In [45]:
import pandas as pd
import json

# Load your dataset (modify the filename accordingly)
file_path = "Data4Good_Arolsen_Archives_50k.csv"  # Change to your actual file path
column_name = "Geo Location"    # Change to your actual column name

# Read the dataset
df = pd.read_csv(file_path)


# Extract the JSON column
geo_data = df[column_name].tolist()

# Save to a JSON file
output_file = "geo_data.json"
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(geo_data, f, ensure_ascii=False, indent=4)

print(f"JSON data saved to {output_file}")


JSON data saved to geo_data.json


In [51]:
import json
import pandas as pd

# Load JSON data from the file
input_file = "geo_data.json"  # Adjust to your actual file
with open(input_file, 'r', encoding='utf-8') as f:
    geo_data = json.load(f)

# Prepare a list to store the transformed rows
rows = []

# Iterate through each JSON object in the dataset
for record_id, record_str in enumerate(geo_data, start=1):
    try:
        # Ensure record_str is treated as a string
        if not isinstance(record_str, str):
            record_str = "{}"  # Replace non-string values with an empty JSON object
        
        # Clean up the JSON string by removing outer quotes and fixing escape sequences
        cleaned_str = record_str.strip('"').replace('""', '"')
        
        # Convert cleaned string into a valid JSON object, or empty if parsing fails
        try:
            record = json.loads(cleaned_str)
        except json.JSONDecodeError:
            record = {}  # Assign an empty dictionary in case of decoding error

        paths = record.get("paths", [])
        markers = {marker.get("label", "Unknown"): marker.get("type", "Unknown") for marker in record.get("markers", [])}
        
        for path in paths:
            origin = path.get("fromLabel", "Unknown")
            dest = path.get("toLabel", "Unknown")
            index = path.get("index", -1)  # Default to -1 if index is missing
            
            # Get the type by matching origin with marker labels
            type_value = markers.get(origin, "Unknown")

            # Append the row to the list
            rows.append({
                "ID": record_id,
                "Origin": origin,
                "Dest": dest,
                "Index": index,
                "Type": type_value
            })
        
        # Handle cases where no valid paths exist (e.g., empty JSONs)
        if not paths:
            rows.append({
                "ID": record_id,
                "Origin": "Unknown",
                "Dest": "Unknown",
                "Index": -1,
                "Type": "Unknown"
            })

    except Exception as e:
        print(f"Unexpected error in record {record_id}: {e}")
        continue

# Convert list to DataFrame
df = pd.DataFrame(rows)

# Save to CSV file
output_file = "transformed_dataset.csv"
df.to_csv(output_file, index=False, encoding='utf-8')

print(f"Structured data saved to {output_file}")


Structured data saved to transformed_dataset.csv
